<h1>Problem description</h1>

When you live in a big city, usually you what to be at home in a neighborhood where you have all main services nearby. The most common services to be comfortable at place where you live are following:
-	Food markets
-	Supermarkets
-	Parks 
-	Shopping malls
-	Grocery stores
-	Government services centers
-	Transport services
Also, when we select a neighborhood to live, we evaluate safety and most nationality of people that live in that place. 

Getting data from Foursquare we will try to predict which neighborhood in the city of Toronto follows criteria mentioned before and represent it in a Map. 

The target audience of this project are citizens and business persons in Toronto, Ontario in Canada. 

<h1>Data collection</h1>


This project will require the exploration and analysis of following data:
-	Toronto Neighborhood Data 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada'  which provides a list of neighborhood in the Toronto area and use of a geo locator to determine the latitude and longitude of each area.
-	Foursquare Location Data to get information and rating about venues for each neighborhood. We will consider this information to cluster target for citizens and make recommendations. 

<h1> Methodology </h1>

Firstly, Toronto borough neighborhood addresses are converted into their equivalent latitude and longitude using the values retrieved from Toronto geodata sheet using Pandas library in Python. 
After that, the Foursquare API is used to explore neighborhoods in Toronto borough. Using the explore function, the most common venue within a radius of 800 meters is extracted for each Toronto neighborhood and the number of returned venues by Foursquare API is limited to 100.
Next, the venue categories feature is used to group the neighborhoods into clusters. The k-means clustering algorithm is used to perform this task.
Finally, the Folium library is employed to visualize the neighborhoods in Toronto borough and their emerging clusters with colors and recommendations for the best place for living.
<h1> Results </h1>
We will get as result a map, with clusters and information of each one related to the rating and the best place for living

<h1> Conclusion </h1>

We would expect that our recommendations will help people or business persons in Toronto, to select a place for living.



<h1> Data Section and preparation </h1>


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [2]:
#load toronto neighborhood data from CSV
import pandas as pd
df_Toronto=pd.read_csv('canada_final.csv')
df_Toronto.drop ('Unnamed: 0', axis=1, inplace=True)
print('Data loaded')

Data loaded


In [3]:
df_Toronto.head ()

,Postcode,Borough,Neighbourhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, M3A",43.753259,-79.329656
1,M4A,North York,Victoria Village,"North York, M4A",43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park","Downtown Toronto, M5A",0.000000,0.000000
3,M6A,North York,"Lawrence Heights,Lawrence Manor","North York, M6A",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,"Queen's Park, M7A",43.664366,-79.392328


<h1> Toronto neighborhood exploration </h1>

In [4]:
# TORONTO EXPLORATION
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_Toronto['Borough'].unique()),
        df_Toronto.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


<h1> Neighborhoods of Toronto </h1>

In [5]:
neighbourhood = df_Toronto['Neighbourhood'].unique()
neighbourhood

array(['Parkwoods', 'Victoria Village', 'Harbourfront,Regent Park',
       'Lawrence Heights,Lawrence Manor', "Queen's Park",
       'Islington Avenue', 'Rouge,Malvern', 'Don Mills North',
       'Woodbine Gardens,Parkview Hill', 'Ryerson,Garden District',
       'Glencairn',
       'Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park',
       'Highland Creek,Rouge Hill,Port Union',
       'Flemingdon Park,Don Mills South', 'Woodbine Heights',
       'St. James Town', 'Humewood-Cedarvale',
       'Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe',
       'Guildwood,Morningside,West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor,Downsview North,Wilson Heights',
       'Thorncliffe Park', 'Adelaide,King,Richmond',
       'Dovercourt Village,Dufferin', 'Scarborough Village',
       'Fairview,Henry Farm,Oriole', 'Northwood Park,

<h1> Map of Toronto according its geographical coordinates </h1>

In [6]:
address = 'Toronto, ON, Canada'
geolocator = Nominatim (user_agent="my-application")
location = geolocator.geocode (address)
latitude = location.latitude 
longitude = location.longitude
print ('The geographical coordinate of Toronto, Canada are: {},{} '.format(latitude,longitude))

The geographical coordinate of Toronto, Canada are: 43.653963,-79.387207 


In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, postCode, borough, neighborhood in zip(df_Toronto['Latitude'], 
                                                     df_Toronto['Longitude'],
                                                     df_Toronto['Postcode'],
                                                     df_Toronto['Borough'], 
                                                     df_Toronto['Neighbourhood']):
    label = '{} in {} {}'.format(neighborhood, borough,postCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

<h2> Toronto neighborhoods without post code </h2>

In [8]:
df_Toronto_wt_Post=pd.DataFrame(df_Toronto)
df_Toronto_wt_Post.drop('Postcode', axis=1, inplace=True)
df_Toronto_wt_Post.head(10)

,Borough,Neighbourhood,Address,Latitude,Longitude
0,North York,Parkwoods,"North York, M3A",43.753259,-79.329656
1,North York,Victoria Village,"North York, M4A",43.725882,-79.315572
2,Downtown Toronto,"Harbourfront,Regent Park","Downtown Toronto, M5A",0.000000,0.000000
3,North York,"Lawrence Heights,Lawrence Manor","North York, M6A",43.718518,-79.464763
4,Queen's Park,Queen's Park,"Queen's Park, M7A",43.664366,-79.392328
5,Etobicoke,Islington Avenue,"Etobicoke, M9A",43.667856,-79.532242
6,Scarborough,"Rouge,Malvern","Scarborough, M1B",43.806686,-79.194353
7,North York,Don Mills North,"North York, M3B",43.745906,-79.352188
8,East York,"Woodbine Gardens,Parkview Hill","East York, M4B",43.706397,-79.309937
9,Downtown Toronto,"Ryerson,Garden District","Downtown Toronto, M5B",43.657162,-79.378937


<h3> Foursquare parameters </h3>

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:

def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    names_split = names.split (',')
    for i in range (len(names_split)):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
        
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
            #print (results)
            # return only relevant information for each nearby venue
            venues_list.append([(
                names_split[i], 
                 latitudes, 
                 longitudes, 
                 v['venue']['name'], 
                 v['venue']['location']['lat'], 
                 v['venue']['location']['lng'],  
                 v['venue']['categories'][0]['name']) for v in results])

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        except:
                nearby_venues =  pd.DataFrame()
            
    return(nearby_venues)

In [11]:
#define Foursquare Credentials and Version
# 
CLIENT_ID = 'DNKGBW0DNDMPBVNWGOWNFJBBNB2JCRA1E55P5HH0GJL15BDT'# your Foursquare ID
CLIENT_SECRET = 'VDK2T3LF4QDMH2ISZOESF0MZPVR0U2ROFPJZMUGP4KHLJO4K' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: DNKGBW0DNDMPBVNWGOWNFJBBNB2JCRA1E55P5HH0GJL15BDT
CLIENT_SECRET:VDK2T3LF4QDMH2ISZOESF0MZPVR0U2ROFPJZMUGP4KHLJO4K


In [69]:
#Now, let's get the top 50 venues that are in each Address a radius of 800 meters.


LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 800 # define radius
df_toronto_venues = pd.DataFrame (columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'])
#regards limitations of the platform we will use only 5% of the dataset
for i in range(round((len(df_Toronto_wt_Post)))):
    neighborhood_latitude = df_Toronto_wt_Post.loc[i, 'Latitude'] # neighborhood latitude value
    neighborhood_longitude = df_Toronto_wt_Post.loc[i, 'Longitude'] # neighborhood longitude value
    
    print (df_Toronto_wt_Post.loc[i, 'Neighbourhood'])
    toronto_venues = getNearbyVenues(names=df_Toronto_wt_Post.loc[i, 'Neighbourhood'],
                                   latitudes=df_Toronto_wt_Post.loc[i, 'Latitude'],
                                   longitudes=df_Toronto_wt_Post.loc[i, 'Longitude']
                                 )
    #print (toronto_venues)
    if len(toronto_venues) > 0:
        df_toronto_venues = df_toronto_venues.append (toronto_venues,ignore_index=True)
df_toronto_venues.head ()

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,DVP at York Mills,43.758899,-79.334099,Road
3,Parkwoods,43.753259,-79.329656,S&A Tile & Flooring Services,43.753909,-79.320656,Furniture / Home Store
4,Parkwoods,43.753259,-79.329656,TTC Stop #09083,43.759655,-79.332223,Bus Stop


<h3> Saving results to CSV </h3>

In [70]:
df_toronto_venues.shape

(5712, 7)

In [71]:
#df_toronto_venues.to_csv('toronto_venues.csv')

<h1> Loading venues of Toronto saved in toronto_venues.csv </h1>

In [12]:
import pandas as pd
df_toronto_venues=pd.read_csv('toronto_venues.csv')
print('Data loaded')

Data loaded


In [13]:
df_toronto_venues.drop ('Unnamed: 0', axis=1, inplace=True)
df_toronto_venues.head ()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,DVP at York Mills,43.758899,-79.334099,Road
3,Parkwoods,43.753259,-79.329656,S&A Tile & Flooring Services,43.753909,-79.320656,Furniture / Home Store
4,Parkwoods,43.753259,-79.329656,TTC Stop #09083,43.759655,-79.332223,Bus Stop


<h2> Let's check how many venues were returned for each neighborhood </h2>

In [14]:
df_toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,50,50,50,50,50,50
Agincourt,19,19,19,19,19,19
Agincourt North,21,21,21,21,21,21
Albion Gardens,13,13,13,13,13,13
Alderwood,16,16,16,16,16,16
Bathurst Manor,21,21,21,21,21,21
Bathurst Quay,27,27,27,27,27,27
Bayview Village,13,13,13,13,13,13
Beaumond Heights,13,13,13,13,13,13


<h2> Let's find out how many unique categories<h2>

In [15]:
print('There are {} uniques categories.'.format(len(df_toronto_venues['Venue Category'].unique())))

There are 288 uniques categories.


<h2>Now, let's analyze Each Neighborhood </h2>

In [21]:
# one hot encoding
df_toronto_venues_onehot = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_toronto_venues_onehot['Neighbourhood'] = df_toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [df_toronto_venues_onehot.columns[-1]] + list(df_toronto_venues_onehot.columns[:-1])
df_toronto_venues_onehot = df_toronto_venues_onehot[fixed_columns]

df_toronto_venues_onehot.head(10)

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,

In [22]:
df_toronto_venues.shape

(5712, 7)

In [23]:
df_toronto_venues_grouped = df_toronto_venues_onehot.groupby('Neighbourhood').mean().reset_index()
df_toronto_venues_grouped.head (10)

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,

In [24]:
df_toronto_venues_grouped.shape

(206, 289)

<h2>Let's print each neighborhood along with the top 5 most common venues of each neighborhood of Toronto </h2>

In [25]:
num_top_venues = 5

for hood in df_toronto_venues_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = df_toronto_venues_grouped[df_toronto_venues_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0  American Restaurant  0.08
1                 Café  0.06
2           Steakhouse  0.06
3                Hotel  0.04
4          Coffee Shop  0.04


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.11
1      Breakfast Spot  0.05
2       Grocery Store  0.05
3     Badminton Court  0.05
4         Supermarket  0.05


----Agincourt North----
                venue  freq
0  Chinese Restaurant  0.14
1         Pizza Place  0.10
2            Pharmacy  0.10
3                Park  0.10
4           BBQ Joint  0.05


----Albion Gardens----
            venue  freq
0     Pizza Place  0.23
1   Grocery Store  0.23
2  Sandwich Place  0.08
3        Pharmacy  0.08
4     Coffee Shop  0.08


----Alderwood----
                venue  freq
0         Pizza Place  0.12
1                Park  0.06
2      Sandwich Place  0.06
3         Gas Station  0.06
4  Athletics & Sports  0.06


----Bathurst Manor----
              venue  freq
0       Pizza Place

In [26]:
#Sort the venues in descending order.


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h1>Display the top 15 venues for each neighborhood </h1>

In [27]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = df_toronto_venues_grouped['Neighbourhood']

for ind in np.arange(df_toronto_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_toronto_venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Adelaide,American Restaurant,Café,Steakhouse,Restaurant,Coffee Shop,Concert Hall,Asian Restaurant,Breakfast Spot,Hotel,General Travel,Seafood Restaurant,Brazilian Restaurant,Lounge,Smoke Shop,Jazz Club
1,Agincourt,Chinese Restaurant,Shopping Mall,American Restaurant,Sushi Restaurant,Restaurant,Malay Restaurant,Motorcycle Shop,Discount Store,Pool Hall,Supermarket,Breakfast Spot,Mediterranean Restaurant,Seafood Restaurant,Grocery Store,Shanghai Restaurant
2,Agincourt North,Chinese Restaurant,Park,Pharmacy,Pizza Place,Coffee Shop,BBQ Joint,Hobby Shop,Fried Chicken Joint,Korean Restaurant,Fast Food Restaurant,Malay Restaurant,Noodle House,Caribbean Restaurant,Bubble Tea Shop,Shop & Service
3,Albion Gardens,Pizza Place,Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Hardware Store,Airport Service,Eastern European Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
4,Alderwood,Pizza Place,Athletics & Sports,Gas Station,Donut Shop,Dance Studio,Convenience Store,Park,Coffee Shop,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Bank,Gym
5,Bathurst Manor,Coffee Shop,Pizza Place,Park,Ice Cream Shop,Eastern European Restaurant,Bridal Shop,Sandwich Place,Diner,Bank,Restaurant,Deli / Bodega,Supermarket,Sushi Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
6,Bathurst Quay,Harbor / Marina,Airport Terminal,Boat or Ferry,Airport Lounge,Airport Service,Sculpture Garden,Park,Coffee Shop,Music Venue,Plane,Rental Car Location,Boutique,Tunnel,Airport Gate,Airport Food Court
7,Bayview Village,Bank,Japanese Restaurant,Restaurant,Chinese Restaurant,Skate Park,Shopping Mall,Café,Convenience Store,Skating Rink,Pharmacy,Grocery Store,Community Center,Dessert Shop,Elementary School,Electronics Store
8,Beaumond Heights,Pizza Place,Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Hardware Store,Airport Service,Eastern European Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
9,Bedford Park,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Skating Rink,Baby Store,Sports Club,Bakery,Bank,Indian Restaurant,Intersection,Juice Bar,Comfort Food Restaurant,Pizza Place,Farmers Market,Pub


In [28]:
df_toronto_venues_grouped.shape

(206, 289)

<h1>Cluster Neighborhoods </h1>

In [29]:
# set number of clusters
kclusters = 6

df_toronto_venues_grouped_clustering = df_toronto_venues_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_toronto_venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 4, 4, 4, 4, 0, 0, 4, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 5, 5, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4,
       4, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0,
       0, 0, 2, 4, 1, 0, 0, 4, 0, 0, 0, 0, 4, 4, 4, 0, 0, 0, 0, 0, 1, 4,
       1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1, 4, 4, 4, 0, 4, 1, 0, 0, 0,
       0, 4, 0, 4, 0, 0, 0, 1, 0, 3, 4, 0, 4, 0, 1, 0, 0, 0, 0, 0, 2, 4,
       0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 4, 5, 0, 4, 0, 0,
       0, 4, 0, 0, 4, 4, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0, 0, 0, 0, 4, 0, 0,
       1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 4,
       4, 0, 0, 0, 0, 0, 4, 0], dtype=int32)

In [30]:
neighborhood_list_geo = df_toronto_venues.groupby('Neighbourhood').mean().reset_index()
neighborhood_list_geo.drop ('Venue Latitude',axis=1, inplace=True)
neighborhood_list_geo.drop ('Venue Longitude',axis=1, inplace=True)
neighborhood_list_geo.head (10)

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude
0,Adelaide,43.650571,-79.384568
1,Agincourt,43.794200,-79.262029
2,Agincourt North,43.815252,-79.284577
3,Albion Gardens,43.739416,-79.588437
4,Alderwood,43.602414,-79.543484
5,Bathurst Manor,43.754328,-79.442259
6,Bathurst Quay,43.628947,-79.394420
7,Bayview Village,43.786947,-79.385975
8,Beaumond Heights,43.739416,-79.588437
9,Bedford Park,43.733283,-79.419750


<h3> Unify the cluster data with Neighborhood Data </h3>

In [31]:
nh_toronto_merged = neighborhood_list_geo

# add clustering labels
nh_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nh_toronto_merged = nh_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

nh_toronto_merged.head() # check the last columns!

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Adelaide,43.650571,-79.384568,0,American Restaurant,Café,Steakhouse,Restaurant,Coffee Shop,Concert Hall,Asian Restaurant,Breakfast Spot,Hotel,General Travel,Seafood Restaurant,Brazilian Restaurant,Lounge,Smoke Shop,Jazz Club
1,Agincourt,43.794200,-79.262029,0,Chinese Restaurant,Shopping Mall,American Restaurant,Sushi Restaurant,Restaurant,Malay Restaurant,Motorcycle Shop,Discount Store,Pool Hall,Supermarket,Breakfast Spot,Mediterranean Restaurant,Seafood Restaurant,Grocery Store,Shanghai Restaurant
2,Agincourt North,43.815252,-79.284577,4,Chinese Restaurant,Park,Pharmacy,Pizza Place,Coffee Shop,BBQ Joint,Hobby Shop,Fried Chicken Joint,Korean Restaurant,Fast Food Restaurant,Malay Restaurant,Noodle House,Caribbean Restaurant,Bubble Tea Shop,Shop & Service
3,Albion Gardens,43.739416,-79.588437,4,Pizza Place,Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Hardware Store,Airport Service,Eastern European Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
4,Alderwood,43.602414,-79.543484,4,Pizza Place,Athletics & Sports,Gas Station,Donut Shop,Dance Studio,Convenience Store,Park,Coffee Shop,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Bank,Gym


In [32]:
nh_toronto_merged.shape

(206, 19)

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1> Examine the Clusters </h1>

<h2> Cluster #1 </h2>

In [34]:
#Examine Clusters
#Cluster 1

nh_toronto_merged.loc[nh_toronto_merged['Cluster Labels'] == 0, nh_toronto_merged.columns[[0] + list(range(4, nh_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Adelaide,American Restaurant,Café,Steakhouse,Restaurant,Coffee Shop,Concert Hall,Asian Restaurant,Breakfast Spot,Hotel,General Travel,Seafood Restaurant,Brazilian Restaurant,Lounge,Smoke Shop,Jazz Club
1,Agincourt,Chinese Restaurant,Shopping Mall,American Restaurant,Sushi Restaurant,Restaurant,Malay Restaurant,Motorcycle Shop,Discount Store,Pool Hall,Supermarket,Breakfast Spot,Mediterranean Restaurant,Seafood Restaurant,Grocery Store,Shanghai Restaurant
6,Bathurst Quay,Harbor / Marina,Airport Terminal,Boat or Ferry,Airport Lounge,Airport Service,Sculpture Garden,Park,Coffee Shop,Music Venue,Plane,Rental Car Location,Boutique,Tunnel,Airport Gate,Airport Food Court
7,Bayview Village,Bank,Japanese Restaurant,Restaurant,Chinese Restaurant,Skate Park,Shopping Mall,Café,Convenience Store,Skating Rink,Pharmacy,Grocery Store,Community Center,Dessert Shop,Elementary School,Electronics Store
9,Bedford Park,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Skating Rink,Baby Store,Sports Club,Bakery,Bank,Indian Restaurant,Intersection,Juice Bar,Comfort Food Restaurant,Pizza Place,Farmers Market,Pub
10,Berczy Park,Café,Bakery,Seafood Restaurant,Farmers Market,Creperie,Cocktail Bar,Steakhouse,Japanese Restaurant,Beer Bar,Basketball Stadium,Restaurant,Sports Bar,Sporting Goods Shop,Fish Market,Italian Restaurant
11,Birch Cliff,College Stadium,Bank,Skating Rink,Park,Thai Restaurant,Café,Diner,Discount Store,General Entertainment,Dessert Shop,Dim Sum Restaurant,Dive Bar,Doner Restaurant,Department Store,Donut Shop
13,Brockton,Café,Coffee Shop,Grocery Store,Bar,Italian Restaurant,Bakery,Arts & Crafts Store,Flea Market,Mexican Restaurant,Antique Shop,Cajun / Creole Restaurant,Sports Bar,Speakeasy,Dessert Shop,Yoga Studio
14,Business reply mail Processing Centre969 Eastern,Fast Food Restaurant,Italian Restaurant,Coffee Shop,Bar,Park,Grocery Store,Pizza Place,Harbor / Marina,Burrito Place,Yoga Studio,Falafel Restaurant,Skate Park,Beer Store,Farmers Market,Snack Place
16,CN Tower,Harbor / Marina,Airport Terminal,Boat or Ferry,Airport Lounge,Airport Service,Sculpture Garden,Park,Coffee Shop,Music Venue,Plane,Rental Car Location,Boutique,Tunnel,Airport Gate,Airport Food Court


In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (cluster == 0):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Cluster #2 </h2>

In [36]:
#Examine Clusters
#Cluster 2
nh_toronto_merged.loc[nh_toronto_merged['Cluster Labels'] == 1, nh_toronto_merged.columns[[0] + list(range(4, nh_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
70,Humber Bay,Construction & Landscaping,Baseball Field,Gym / Fitness Center,Park,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio,Design Studio,Eastern European Restaurant,Electronics Store
86,King's Mill Park,Construction & Landscaping,Baseball Field,Gym / Fitness Center,Park,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio,Design Studio,Eastern European Restaurant,Electronics Store
88,Kingsway Park South East,Construction & Landscaping,Baseball Field,Gym / Fitness Center,Park,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio,Design Studio,Eastern European Restaurant,Electronics Store
100,Maple Leaf Park,Construction & Landscaping,Tennis Court,Park,Trail,Bakery,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Yoga Studio,Department Store,Donut Shop,Dumpling Restaurant
106,Mimico NE,Construction & Landscaping,Baseball Field,Gym / Fitness Center,Park,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio,Design Studio,Eastern European Restaurant,Electronics Store
117,North Park,Construction & Landscaping,Tennis Court,Park,Trail,Bakery,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Yoga Studio,Department Store,Donut Shop,Dumpling Restaurant
124,Old Mill South,Construction & Landscaping,Baseball Field,Gym / Fitness Center,Park,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio,Design Studio,Eastern European Restaurant,Electronics Store
143,Royal York South East,Construction & Landscaping,Baseball Field,Gym / Fitness Center,Park,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio,Design Studio,Eastern European Restaurant,Electronics Store
166,Sunnylea,Construction & Landscaping,Baseball Field,Gym / Fitness Center,Park,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio,Design Studio,Eastern European Restaurant,Electronics Store
176,The Queensway East,Construction & Landscaping,Baseball Field,Gym / Fitness Center,Park,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio,Design Studio,Eastern European Restaurant,Electronics Store


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (cluster == 1):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Cluster #3 </h2>

In [38]:
#Examine Clusters
#Cluster 3

nh_toronto_merged.loc[nh_toronto_merged['Cluster Labels'] == 2, nh_toronto_merged.columns[[0] + list(range(4, nh_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
68,Highland Creek,Breakfast Spot,Bar,Italian Restaurant,Burger Joint,Dumpling Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop,Eastern European Restaurant,Electronics Store,Elementary School
130,Port Union,Breakfast Spot,Bar,Italian Restaurant,Burger Joint,Dumpling Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop,Eastern European Restaurant,Electronics Store,Elementary School
142,Rouge Hill,Breakfast Spot,Bar,Italian Restaurant,Burger Joint,Dumpling Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop,Eastern European Restaurant,Electronics Store,Elementary School


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (cluster == 2):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Cluster #4 </h2>

In [40]:
#Examine Clusters
#Cluster 4

nh_toronto_merged.loc[nh_toronto_merged['Cluster Labels'] == 3, nh_toronto_merged.columns[[0] + list(range(4, nh_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
119,Northwest,Lounge,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Dumpling Restaurant,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant


In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (cluster == 3):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Cluster #5 </h2>

In [42]:
#Examine Clusters
#Cluster 5

nh_toronto_merged.loc[nh_toronto_merged['Cluster Labels'] == 4, nh_toronto_merged.columns[[0] + list(range(4, nh_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
2,Agincourt North,Chinese Restaurant,Park,Pharmacy,Pizza Place,Coffee Shop,BBQ Joint,Hobby Shop,Fried Chicken Joint,Korean Restaurant,Fast Food Restaurant,Malay Restaurant,Noodle House,Caribbean Restaurant,Bubble Tea Shop,Shop & Service
3,Albion Gardens,Pizza Place,Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Hardware Store,Airport Service,Eastern European Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
4,Alderwood,Pizza Place,Athletics & Sports,Gas Station,Donut Shop,Dance Studio,Convenience Store,Park,Coffee Shop,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Bank,Gym
5,Bathurst Manor,Coffee Shop,Pizza Place,Park,Ice Cream Shop,Eastern European Restaurant,Bridal Shop,Sandwich Place,Diner,Bank,Restaurant,Deli / Bodega,Supermarket,Sushi Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
8,Beaumond Heights,Pizza Place,Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Hardware Store,Airport Service,Eastern European Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
12,Bloordale Gardens,Pizza Place,Café,Beer Store,Liquor Store,Electronics Store,Pharmacy,Coffee Shop,Park,Comfort Food Restaurant,Dim Sum Restaurant,Ethiopian Restaurant,Empanada Restaurant,Elementary School,College Theater,Comedy Club
15,CFB Toronto,Coffee Shop,Pizza Place,Vietnamese Restaurant,Sandwich Place,Latin American Restaurant,Turkish Restaurant,Middle Eastern Restaurant,Airport,Park,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Donut Shop,Doner Restaurant
26,Clarks Corners,Pharmacy,Pizza Place,Park,Discount Store,Italian Restaurant,Intersection,Shopping Mall,Fried Chicken Joint,Thai Restaurant,Sandwich Place,Fast Food Restaurant,Noodle House,Chinese Restaurant,Seafood Restaurant,Coffee Shop
30,Cloverdale,Pizza Place,Bank,Hotel,Convenience Store,American Restaurant,Café,Clothing Store,Burrito Place,Theater,Doner Restaurant,Diner,Discount Store,Dive Bar,Yoga Studio,Donut Shop
42,Downsview East,Coffee Shop,Pizza Place,Vietnamese Restaurant,Sandwich Place,Latin American Restaurant,Turkish Restaurant,Middle Eastern Restaurant,Airport,Park,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Donut Shop,Doner Restaurant


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (cluster == 4):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Cluster #6 </h2>

In [45]:
#Examine Clusters
#Cluster 6

nh_toronto_merged.loc[nh_toronto_merged['Cluster Labels'] == 5, nh_toronto_merged.columns[[0] + list(range(4, nh_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
27,Cliffcrest,Pizza Place,Ice Cream Shop,Furniture / Home Store,Park,Burger Joint,Dive Bar,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Deli / Bodega,Donut Shop,Dumpling Restaurant
28,Cliffside,Pizza Place,Ice Cream Shop,Furniture / Home Store,Park,Burger Joint,Dive Bar,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Deli / Bodega,Donut Shop,Dumpling Restaurant
149,Scarborough Village West,Pizza Place,Ice Cream Shop,Furniture / Home Store,Park,Burger Joint,Dive Bar,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Deli / Bodega,Donut Shop,Dumpling Restaurant


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (cluster == 5):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Best neighborhood for living</h2>

<h1> Has a park? </h1>

In [52]:
nh_toronto_merged.loc[nh_toronto_merged['1st Most Common Venue']=='Park']

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
18,Caledonia-Fairbanks,43.689026,-79.453512,0,Park,Pharmacy,Japanese Restaurant,Gym,Pizza Place,Cosmetics Shop,Café,Discount Store,Mexican Restaurant,Market,Fast Food Restaurant,Beer Store,Sporting Goods Shop,Bank,Bakery
69,Hillcrest Village,43.803762,-79.363452,4,Park,Pharmacy,Housing Development,Sandwich Place,Diner,Korean Restaurant,Bank,Bakery,Residential Building (Apartment / Condo),Shopping Mall,Ice Cream Shop,Recreation Center,Chinese Restaurant,Coffee Shop,Pizza Place
71,Humber Bay Shores,43.605647,-79.501321,0,Park,Pharmacy,Mexican Restaurant,Sushi Restaurant,Fried Chicken Joint,Elementary School,Sandwich Place,Liquor Store,Falafel Restaurant,Fast Food Restaurant,Dessert Shop,Bus Stop,Fish & Chips Shop,Flower Shop,Bakery
108,Mimico South,43.605647,-79.501321,0,Park,Pharmacy,Mexican Restaurant,Sushi Restaurant,Fried Chicken Joint,Elementary School,Sandwich Place,Liquor Store,Falafel Restaurant,Fast Food Restaurant,Dessert Shop,Bus Stop,Fish & Chips Shop,Flower Shop,Bakery
114,New Toronto,43.605647,-79.501321,0,Park,Pharmacy,Mexican Restaurant,Sushi Restaurant,Fried Chicken Joint,Elementary School,Sandwich Place,Liquor Store,Falafel Restaurant,Fast Food Restaurant,Dessert Shop,Bus Stop,Fish & Chips Shop,Flower Shop,Bakery
139,Rosedale,43.679563,-79.377529,0,Park,Bank,Trail,Playground,Grocery Store,Candy Store,Yoga Studio,Dive Bar,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Doner Restaurant,Department Store


In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for criteria, lat, lon, poi, cluster in zip(nh_toronto_merged['1st Most Common Venue'],
                                    nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (criteria == 'Park'):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>Has a Elementary School?</h1>

In [63]:
nh_toronto_merged.loc[nh_toronto_merged['6th Most Common Venue']=='Elementary School']

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
71,Humber Bay Shores,43.605647,-79.501321,0,Park,Pharmacy,Mexican Restaurant,Sushi Restaurant,Fried Chicken Joint,Elementary School,Sandwich Place,Liquor Store,Falafel Restaurant,Fast Food Restaurant,Dessert Shop,Bus Stop,Fish & Chips Shop,Flower Shop,Bakery
108,Mimico South,43.605647,-79.501321,0,Park,Pharmacy,Mexican Restaurant,Sushi Restaurant,Fried Chicken Joint,Elementary School,Sandwich Place,Liquor Store,Falafel Restaurant,Fast Food Restaurant,Dessert Shop,Bus Stop,Fish & Chips Shop,Flower Shop,Bakery
114,New Toronto,43.605647,-79.501321,0,Park,Pharmacy,Mexican Restaurant,Sushi Restaurant,Fried Chicken Joint,Elementary School,Sandwich Place,Liquor Store,Falafel Restaurant,Fast Food Restaurant,Dessert Shop,Bus Stop,Fish & Chips Shop,Flower Shop,Bakery
192,Weston,43.706876,-79.518188,4,Pizza Place,Breakfast Spot,Café,Fried Chicken Joint,Pharmacy,Elementary School,Electronics Store,Eastern European Restaurant,Empanada Restaurant,Dumpling Restaurant,Department Store,Doner Restaurant,Dive Bar,Discount Store,Diner


In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for criteria, lat, lon, poi, cluster in zip(nh_toronto_merged['6th Most Common Venue'],
                                    nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (criteria == 'Elementary School'):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1> Has a Mall? </h1>

In [78]:
nh_toronto_merged.loc[nh_toronto_merged['7th Most Common Venue']=='Shopping Mall']

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
26,Clarks Corners,43.781637,-79.304302,4,Pharmacy,Pizza Place,Park,Discount Store,Italian Restaurant,Intersection,Shopping Mall,Fried Chicken Joint,Thai Restaurant,Sandwich Place,Fast Food Restaurant,Noodle House,Chinese Restaurant,Seafood Restaurant,Coffee Shop
49,Emery,43.724766,-79.532242,0,Convenience Store,Home Service,Italian Restaurant,Discount Store,Baseball Field,Furniture / Home Store,Shopping Mall,Elementary School,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Design Studio,Donut Shop,Doner Restaurant,Dive Bar
74,Humberlea,43.724766,-79.532242,0,Convenience Store,Home Service,Italian Restaurant,Discount Store,Baseball Field,Furniture / Home Store,Shopping Mall,Elementary School,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Design Studio,Donut Shop,Doner Restaurant,Dive Bar
163,Sullivan,43.781637,-79.304302,4,Pharmacy,Pizza Place,Park,Discount Store,Italian Restaurant,Intersection,Shopping Mall,Fried Chicken Joint,Thai Restaurant,Sandwich Place,Fast Food Restaurant,Noodle House,Chinese Restaurant,Seafood Restaurant,Coffee Shop
168,Tam O'Shanter,43.781637,-79.304302,4,Pharmacy,Pizza Place,Park,Discount Store,Italian Restaurant,Intersection,Shopping Mall,Fried Chicken Joint,Thai Restaurant,Sandwich Place,Fast Food Restaurant,Noodle House,Chinese Restaurant,Seafood Restaurant,Coffee Shop


In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for criteria, lat, lon, poi, cluster in zip(nh_toronto_merged['7th Most Common Venue'],
                                    nh_toronto_merged['Neighborhood Latitude'], 
                                  nh_toronto_merged['Neighborhood Longitude'], 
                                  nh_toronto_merged['Neighbourhood'], 
                                  nh_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (criteria == 'Shopping Mall'):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters